# CS202: Compiler Construction

## In-class Exercises, Week of 04/24/2023

----

# Lexical and Dynamic Scope

## Question 1

What is the output of this program under *lexical scope*?

In [1]:
z = 1
def f():
    z = 3
    def g():
        print(z)
    return g

z = 5
f()()

3


3, because we used the environment that existed when g was defined, not when it was called.

## Question 2

What would the output be under *dynamic scope*?

5, because we used the env that existed when it was called.

## Question 3

What is the difference between lexical and dynamic scope?

When evaluating a function body when the function is called:

- Lexical scope uses the environment that existed when the function was defined
- Dynamic scope uses the environment that existed when the function was called.

When writing an interpreter, lexical scope is harded to implement than dynamic scope. Dynamic scope is "natural" when writing an interpreter.

## Question 4

What is the output of the following code?

In [2]:
for i in range(10):
    i = i + 1
print(i)

10


The ouput is 10 (not an error message). When a "new scope" starts in python, it does lexical scope, but when saving the environment, it saves the pointer to the environment, not the environment itself. So when we call f, it uses the environment that existed when f was defined, not when it was called.

----
# Closure Conversion

## Question 5

Transform the following program into an `Llambda` program by performing closure conversion.

```
z = 1
def f():
    z = 3
    def g():
        print(z)
    return g

z = 5
f()()
```

In [3]:
z = 1
def f_fun(closure):
    z = 3
    def g_fun(closure):
        z = closure[1]
        print(z)
    g = (g_fun, z)
    return g
f = (f_fun,)
z = 5

tmp1 = f[0](f)
tmp1[0](tmp1)

3


## Question 6

Describe the steps of closure conversion.

1. Modify the function definition to construct a close representation
  - Our representation is a Tuple
  - First value is a pointer to the function itself
  - Later values are the values of the free variables from the function definition
2. Modify the function itself to take the closue representation as an argunemnt and initiliaze closed-over variables  to the value they have in the closure representation
  - Rename the function definition
  - Change the arguments of the function to include the closure representation
3. Modify the function calls of the form f(args...) to f[0](f, args...)

## Question 7

Describe the changes to the compiler required to implement closure conversion.

Add one new pass after the second typechecker called `convert_to_closures`

- Similar to RCO
- Performs the transformation from Q6
- In cc_stmt, for the function definition case, do the transformation descripbed above
  - Find the free variables of the function definition
  - Modify the function def (1st stmt)
  - Construct the closure representation (2nd stmt)
  - One solution: modify global list of functions that you know new name of the function is a function
- In cc_expr, for the function call case, do the transformation described above
  - Modify all functions calls to use a closure representation (even if they have no free variables)
  

Other changes:

- Implement lexical scope in the typecheker (happens by default)
- Before typechecker stores the names of all the functions
- After: functions will be represented by closures

Select instructions:
- Variable case: Assign(x, Var(y)) when y is a function
  - Before: turned into leaq instruction

----
# Dynamic Typing

## Question 8

Describe the challenge of compiling dynamically-typed programs into x86 assembly.

In our current compiler, we rely on types in important ways:
1. We disallow certain operations (e.g. add a number to a function)
2. We produce different code depending on types (e.g for tuple variables or for functions)

In a dynamically typed language:
1. We might not know a variable's type at compile time
2. In our current compiler, no type information attached to values at runtime.

## Question 9

Describe an approach for compiling a dynamically-typed language `Ldyn` to x86 assembly.

Introduce a gradual type languge `Lany`: In a gradually typed language, annotations are optional, and the `any` type can represent any type in the language. Python is a gradually-typed language (now).

Compile `Ldyn` to `Lany` into x86 assembly code. The `Lany` language has the new type `any`, which represents any type.

1. The `any` type  is the type of a *tagged value*. The value itself will have a tag attached to it that says *at runtime* what the type of the value is. (e.g we can use the tag to distinguish between a int and a bool)
2. We introduce `inject` to convert a statically typed value into a value of type `any`. When it does the conversion, it adds the tag indicating that the tagged value has the statically-known type of the input.
3. We introduce `project` to convert a value of type `any` into a statically typed value with a desired type. When it does the conversion `project` (1) checks at runtime that the tag on the input value matches the desied type, and exits if not; (2) removes the tag.

Process:
1. We compile `Ldyn` to `Lany` by adding `inject` and `project` operations.
2. For each constant: use `inject` to convert the constant into a tagged `any` value.
3. For each primitive: use `project` to convert the input to the correct expected types; use `inject` to convert the output to `any`.

After we do this, we can actually run the static typechecker (after adding `any` as a type). Every variable will have the type `any`, and the program will typecheck.

## Question 10

Transform the following `Ldyn` program into an `Lany` program using `inject` and `project`.

```py
x = 5
y = 6
z = x + y
```

In [3]:
from typing import TypeVar
T = TypeVar('T')

from dataclasses import dataclass
@dataclass
class AnyVal:
    val: any
    tag: type

def inject(val: T) -> AnyVal:
    return AnyVal(val, type(val))

def project(tagged_val: AnyVal, t: T) -> T:
    if tagged_val.tag == t:
        return tagged_val.val
    else:
        raise Exception('run-time type error!')

In [5]:
x = inject(5)
y = inject(6)
tmp1 = project(x, int)
tmp2 = project(y, int)
tmp3 = tmp1 + tmp2
z = inject(tmp3)
z

AnyVal(val=11, tag=<class 'int'>)

## Question 11

Describe the changes to the compiler for dynamic typing.

Add the Any type to the typechecker (add as an AnyT dataclass); add typing for inject and project.

Two new passes:
1. `cast_insert`: adds the casts above (inject and project). Put it before the typecheker (because you can't typecheck a program that is dynamically typed). The output of this pass is `Lany`
2. `reveal_casts` compiles the casts into lower-level primitives. Put it after RCO, because it introduces new control flow.

Structure of both passes is similar to RCO.

The reveal casts pass compiles project and inject:
1. Project compiles into an if statement that checks if the tag is correct and returns the value if so; otherwise it exits the program.
2. Inject compiles into the `make_any` primitive that attaches a tag (select instructions will compile it further)

For part #1: use two primitives: `tag_of_any` and `value_of`
- `tag_of` returns the tag of a vaue of type `Any`
- `value_of` returns the value of a value of type `Any`

For `x = project(y, int)`:
We produce 

```py
if tag_of(y) == 1:
  x = value_of(y)
else:
  exit()
```

Can calculate the tag value at compile time (use the tag from section 10.2 in the book; convert to decimal values). We will deal with the three remaining new primitives in select instructions.


## Question 12

Write assembly language instructions for the following `Cany` statements. Reference Section 10.9 in the textbook.

1. `x = make_any(5, 1)`
2. `tmp_4 = tag_of(x)`
3. `tmp_1 = value_of(x)`

We will compile these into assembly instructions in the select instructions pass (one new case in si_stmt per primitive).

1. `make_any` adds the tag: shifts the value 3 bits to the left, then adds the tag to the value.

`x = make_any(5, 1)`

```
salq $3, #x -- shift left by 3 bits
orq $1, #x -- add the tag
```

2. `tag_of` gets JUST the tag piece of a tagged value

`tmp_4 = tag_of(x)`

```
movq #x, %tmp_4 -- copy the tagged value to the destination
addq $7, %tmp_4 -- erase everything except the tag (7 = 0b111)
```

3. `value_of` gets JUST the value piece of a tagged value

`tmp_1 = value_of(x)`

```
movq #x, %tmp_1 -- copy the tagged value to the destination
sarq $3, %tmp_1 -- shift right by 3 bits, erasing the tag
```

For the exit primitive, you can jump to `main_conclusion` (which is the end of the program).

## Question 13

What impact will these changes have on the performance of our compiled code? What would happen if we skipped this new code?

Think about a simple addition: `x = a + b`

In our statically-typed language, this turns into a single addq instruction.

In the dynamically-typed language, it turns into:
- At least two if statement
- At least three shifts

Each of these extra operations adds overhead. What was 1 instruction now might be more like 10-20 instructions. This has a huge impact on performance, sometimes >10x.

This is one reason dynamically-typed programing languages have a reputation for being slow. (the other is that they are often implemented using interpreters).

This seems super bad! One implications is that you can view typing as a performance optimization.

Options:
1. skip checks. Outcome can be: operate on the byte-level representation of a value, while considering it to represent the wrong type. (e.g. f + 5 where f is a function) This is basically what C does.
2. Use a fancy compiler, like a just-in-time compiler (JIT). A JIT compiles code to native x86 assembly *at runtime*. The reason is that at runtime, you know the values of all inputs to a piece of code. That means that you also know the types.

So you can produce code without run-time checks, even for a dynamically-typed language.